In [5]:
# Install necessary packages
%pip install pandas
%pip install datasets
%pip install torch
%pip install transformers -U
%pip install numpy
%pip install -U scikit-learn
%pip install sentencepiece
%pip install tokenizers==0.14.1
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tokenizers-0.14.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.1
    Uninstalling tokenizers-0.11.1:
      Successfully uninstalled tokenizers-0.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum-graphcore 0.5.0 requires transformers==4.20.1, but you have transformers 4.34.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use 

In [8]:
import os
import re
import random
import string
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import nltk
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from torch.utils.tensorboard import SummaryWriter
from transformers import TrainingArguments, Trainer, AdamW, get_cosine_schedule_with_warmup
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score
from transformers import EvalPrediction

In [10]:

def get_and_process_dataset(lang, dataset_type):
    # Set the directory paths
    xml_dir = f"./dataset/pan15-{dataset_type}/pan15-{dataset_type}-{lang}"
    truth_file = f"./dataset/pan15-{dataset_type}/pan15-{dataset_type}-{lang}/truth.txt"
    # Read the truth file
    truth_df = pd.read_csv(truth_file, sep=":::", names=["id", "gender", "age", "extroverted", "stable", "agreeable", "open", "conscientious"])

    # Create an empty dataframe to store the XML data
    xml_data = {"id":[], "all_user_tweets":[]}
    xml_df = pd.DataFrame(xml_data)

    # Iterate through all XML files in the directory
    for filename in os.listdir(xml_dir):
        if filename.endswith(".xml"):
            file_path = os.path.join(xml_dir, filename)

            # Parse the XML file
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Extract the author ID and all user tweets text
            author_id = root.attrib["id"]
            all_user_tweets = []
            for elem in root.iter("document"):
                user_tweet = re.sub(r'\s+', ' ', elem.text.strip())
                user_tweet = user_tweet.replace('\n', '').replace('\t', '')
                user_tweet = re.sub(r'[^\w\s.,?!$\'"@\\]', '', user_tweet)
                
                all_user_tweets.append(user_tweet)

            # Add the data to the xml dataframe
            xml_df = xml_df.append({"id": author_id, "all_user_tweets": "\n ".join(all_user_tweets)}, ignore_index=True)

    # Merge the XML data and the truth data into a single dataframe
    merged_df = pd.merge(truth_df, xml_df, on="id")

    # Reorder the columns as specified
    return merged_df[["gender", "age", "extroverted", "stable", "agreeable", "open", "conscientious", "all_user_tweets"]]

In [11]:
train_dataset = pd.read_csv('en_df')
test_dataset = pd.read_csv('test_df')

In [85]:
train_dataset = get_and_process_dataset("english", "train")
test_dataset = get_and_process_dataset("english", "test")

<ipython-input-84-c4c851446175>:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  truth_df = pd.read_csv(truth_file, sep=":::", names=["id", "gender", "age", "extroverted", "stable", "agreeable", "open", "conscientious"])
<ipython-input-84-c4c851446175>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xml_df = xml_df.append({"id": author_id, "all_user_tweets": "\n ".join(all_user_tweets)}, ignore_index=True)
<ipython-input-84-c4c851446175>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xml_df = xml_df.append({"id": author_id, "all_user_tweets": "\n ".join(all_user_tweets)}, ignore_index=True)
<ipython-input-84-c4c85

In [12]:
train_dataset

,gender,age,extroverted,stable,agreeable,open,conscientious,all_user_tweets
0,M,25-34,0.3,0.5,0.1,0.2,0.2,How to Test Your Startup Idea for $50 httpt.co...
1,M,25-34,0.2,0.4,0.2,0.0,0.2,Everyday I come up with a new optimum solution...
2,M,18-24,0.1,0.2,0.1,0.1,0.1,@username @username @username @username IfMyNa...
3,F,18-24,0.1,0.1,-0.1,0.5,0.1,I wish we could talk forever.\n 100000 things ...
4,F,18-24,0.0,-0.1,0.1,0.3,0.4,icarly \n thuis3\n Homework finished.\n morgen...
...,...,...,...,...,...,...,...,...
147,M,25-34,0.2,0.2,0.1,0.1,0.5,"""If you fight for a cause then fight for a cau..."
148,F,18-24,0.5,0.3,0.0,0.1,0.3,We ain't picture perfect but we worth the pict...
149,F,25-34,0.4,-0.3,0.0,0.1,0.3,Nasa to turn ISS into perfect Earthobserving p...
150,M,25-34,0.3,0.5,0.1,0.2,0.2,lisent @username Nadim talks to @username abou...


In [17]:
def preprocess_df(df):
    # Reset the index of the dataframe
    df = df.reset_index(drop=True)
    
    # Create a copy of the original dataframe to avoid modifying it
    df_copy = df.copy()
    
    # Map gender values to 0 or 1
    df_copy['gender'] = df_copy['gender'].map({'M': 0, 'F': 1})
    df_copy['age'] = df_copy['age'].map({'18-24': 0, '25-34':1, '35-49': 1, '50-XX': 1})
    
    # Rename the 'all_user_tweets' column to 'text'
    # df_copy = df_copy.rename(columns={'all_user_tweets': 'text'})
    
    # Convert all tweets to strings
    df_copy['text'] = df_copy['all_user_tweets'].astype(str)
    
    # Convert label columns to numeric and binarize the values
    label_cols = ["gender", "extroverted", "stable", "agreeable", "open", "conscientious"]
    df_copy[label_cols] = df_copy[label_cols].apply(pd.to_numeric).applymap(lambda x: int(x > 0))
    
    return df_copy


In [18]:
def preprocess_text(text, num_sentences=70):        
    d = '\n'
    sentences = [s.strip()+d for s in text.split('\n')]

    # select random subset of sentences
    num_sentences = min(num_sentences, len(sentences))
    selected_sentences = random.sample(sentences, num_sentences)

    # clean selected sentences
    text1 = ' '.join(selected_sentences)
    # text1 = re.sub('[%s]' % re.escape(string.punctuation), ' ', text1)
    text1 = re.sub('\w*\d\w*', ' ', text1)
    # text1 = re.sub('<.*?>+', ' ', text1)
    # text1 = re.sub('[^A-Za-z0-9]+', ' ', text1)
    text1 = re.sub(' +', ' ', text1) # remove extra spaces

    return text1

In [19]:
train_df = preprocess_df(train_dataset)
test_df = preprocess_df(test_dataset)

In [20]:
train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)
test_df

,gender,age,extroverted,stable,agreeable,open,conscientious,all_user_tweets,text
0,1,1,1,1,1,1,1,"@username @username Well, just because today i...","THX RT, @username! For all information on the ..."
1,0,1,1,1,1,1,1,I love being an advertiser for @username! I am...,Idly wondering whether httpbit.lycFaQIp mighno...
2,0,0,1,1,1,0,1,"Aight leggo\n Sittin' Sidewayz, instant classi...",Fuck this.\n RT @username ThisCouldBeUsButYouP...
3,0,1,1,1,1,1,1,"I posted 4 photos on Facebook in the album ""Ro...",packing backpack for a trip to Poland travelin...
4,1,1,1,0,0,1,1,The days we don't want to end go by the quicke...,That isn't Olympus has fallen! Olympus e haff ...
...,...,...,...,...,...,...,...,...,...
137,0,1,1,1,1,1,1,Time...commemorative issue httpt.coDU7SaGi\n @...,Live from new ...amazing!!!!! apple App_iOS\n ...
138,0,1,0,1,1,1,1,Second International Workshop on Privacy and S...,Judith Bishop of Microsoft Research visited II...
139,0,1,0,1,1,1,1,Dev leaves minor bug in App User gives 1star ...,@username add Flatland by Abbott to your next ...
140,1,0,1,1,1,1,1,I really feel that I'm losing my best friend.....,Baby come back ????\n I love this feeling that...


In [21]:
from datasets import Dataset

cols = ["gender", "age", "extroverted", "stable", "agreeable", "open", "conscientious", "text"]
labels = ["gender", "age", "extroverted", "stable", "agreeable", "open", "conscientious"]
train_dt = Dataset.from_pandas(train_df[cols])
test_dt = Dataset.from_pandas(test_df[cols])
train_dt

Dataset({
    features: ['gender', 'age', 'extroverted', 'stable', 'agreeable', 'open', 'conscientious', 'text'],
    num_rows: 152
})

## Training and inference with classical models

In [24]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
# from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [26]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
SVC_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(LinearSVC())),
])

LR_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='lbfgs', max_iter=1000))),
])

In [28]:
from sklearn.metrics import (accuracy_score, 
                            precision_score, 
                            recall_score, 
                            f1_score, 
                            roc_auc_score)

In [29]:
# Train the SVC model
SVC_pipeline.fit(train_df['text'], train_df[labels_col])

# Get the decision scores for the test set
y_scores = SVC_pipeline.decision_function(test_df['text'])

# Adjust the threshold for positive classification
new_threshold = -0.0# Example threshold value
y_pred = (y_scores > new_threshold).astype(int)

# Evaluate the model
average='weighted'
precision = precision_score(test_df[labels_col], y_pred, average=average)
recall = recall_score(test_df[labels_col], y_pred, average=average)
f1 = f1_score(test_df[labels_col], y_pred, average=average)
roc_auc = roc_auc_score(test_df[labels_col], y_scores, average=average)

print('Multi-label precision score for SVC :', precision)
print('Multi-label recall score for SVC :', recall)
print('Multi-label F1 score for SVC :', f1)
print('Multi-label ROC-AUC score for SVC :', roc_auc)

Multi-label precision score for SVC : 0.8306412169977089
Multi-label recall score for SVC : 0.9779179810725552
Multi-label F1 score for SVC : 0.8961758289703985
Multi-label ROC-AUC score for SVC : 0.8620882333779316


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_classes.py:32: FutureWar

In [30]:
# Fit the pipeline on training data
NB_pipeline.fit(train_df['text'], train_df[labels_col])

# Predict using the NB model
y_pred_NB = NB_pipeline.predict(test_df['text'])

# Evaluate the NB model
precision_NB = precision_score(test_df[labels_col], y_pred_NB, average=average)
recall_NB = recall_score(test_df[labels_col], y_pred_NB, average=average)
f1_NB = f1_score(test_df[labels_col], y_pred_NB, average=average)

print('Multi-label precision score for Naive Bayes:', precision_NB)
print('Multi-label recall score for Naive Bayes:', recall_NB)
print('Multi-label F1 score for Naive Bayes:', f1_NB)

Multi-label precision score for Naive Bayes: 0.7897708512471502
Multi-label recall score for Naive Bayes: 0.9936908517350158
Multi-label F1 score for Naive Bayes: 0.8763609419444536


In [31]:
# Train the LR model
LR_pipeline.fit(train_df['text'], train_df[labels_col])

# Predict using the LR model
y_pred_LR = LR_pipeline.predict(test_df['text'])

# Evaluate the LR model
precision_LR = precision_score(test_df[labels_col], y_pred_LR, average=average)
recall_LR = recall_score(test_df[labels_col], y_pred_LR, average=average)
f1_LR = f1_score(test_df[labels_col], y_pred_LR, average=average)

print('Multi-label precision score for Logistic Regression:', precision_LR)
print('Multi-label recall score for Logistic Regression:', recall_LR)
print('Multi-label F1 score for Logistic Regression:', f1_LR)

Multi-label precision score for Logistic Regression: 0.7913240000130041
Multi-label recall score for Logistic Regression: 0.9747634069400631
Multi-label F1 score for Logistic Regression: 0.8701330744062953


In [32]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 20.5 MB/s eta 0:00:0000:0100:01


In [34]:
import xgboost as xgb


XGB_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(xgb.XGBClassifier())),
])

# Fit the pipeline on training data
XGB_pipeline.fit(train_df['text'], train_df[labels_col])

# Make predictions on test data
y_pred = XGB_pipeline.predict(test_df['text'])


average='weighted'
precision = precision_score(test_df[labels_col], y_pred, average=average)
recall = recall_score(test_df[labels_col], y_pred, average=average)
f1 = f1_score(test_df[labels_col], y_pred, average=average)
roc_auc = roc_auc_score(test_df[labels_col], y_pred, average=average)

print('Multi-label precision score:', precision)
print('Multi-label recall score:', recall)
print('Multi-label F1 score:', f1)
print('Multi-label ROC-AUC score:', roc_auc)

Multi-label precision score: 0.8111842983321463
Multi-label recall score: 0.9242902208201893
Multi-label F1 score: 0.8631557365239968
Multi-label ROC-AUC score: 0.5640517873634823


## Setup and Training with Language Model ( Roberta)

In [12]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
id2label

{0: 'gender',
 1: 'age',
 2: 'extroverted',
 3: 'stable',
 4: 'agreeable',
 5: 'open',
 6: 'conscientious'}

In [13]:
%pip install tokenizers==0.11.1
from transformers import AutoTokenizer, AutoModelForSequenceClassification


num_labels=7
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base",
                                                            num_labels=num_labels, problem_type="multi_label_classification", 
                                                            id2label=id2label,
                                                            label2id=label2id,
                                                            ignore_mismatched_sizes=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
Note: you may need to restart the kernel to use updated packages.


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weig

In [14]:
import numpy as np
def preprocess_data(examples):
    text = examples["text"]
    encoding = tokenizer(text, padding=True, truncation=True)
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    labels_matrix = np.zeros((len(text), len(labels)))
    for idx, label in enumerate(labels):
        labels_matrix[:, idx]= labels_batch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [15]:
encoded_train = train_dt.map(preprocess_data, batched=True, remove_columns=train_dt.column_names)
encoded_test = test_dt.map(preprocess_data, batched=True, remove_columns=test_dt.column_names)

Map:   0%|          | 0/1368 [00:00<?, ? examples/s]

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

In [16]:
batch_size = 8
metric_name = "f1_weighted"

In [17]:
%pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 67.1 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 74.6 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 36.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
from transformers import TrainingArguments, Trainer, AdamW, get_scheduler
from torch.utils.tensorboard import SummaryWriter

num_train_epochs=6

# Initialize SummaryWriter
writer = SummaryWriter()


from transformers import get_cosine_with_hard_restarts_schedule_with_warmup

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir="./logs",
    logging_steps=265,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    warmup_steps=500,
)

In [19]:
from sklearn.metrics import (f1_score, precision_score,
                             recall_score, roc_auc_score)
import numpy as np
import torch
from transformers import EvalPrediction

def multi_label_metrics(predictions, labels, threshold=0.5):
    # First, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    
    # Next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    
    # Finally, compute metrics
    y_true = labels
    
    metrics = {
        'precision_micro': precision_score(y_true, y_pred, average='micro'),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_micro': recall_score(y_true, y_pred, average='micro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_micro': f1_score(y_true=y_true, y_pred=y_pred, average='micro'),
        'f1_macro': f1_score(y_true=y_true, y_pred=y_pred, average='macro'),
        'f1_weighted': f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    }
    
    return metrics

In [20]:
from transformers import AdamW, get_cosine_schedule_with_warmup

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds, labels=p.label_ids)
    # result["eval_precision"] = result["f1_weighted"]
    return result

num_training_steps = len(encoded_train) // (batch_size * num_train_epochs)
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=training_args.warmup_steps, num_training_steps=num_training_steps, num_cycles=0.5)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler), # pass the scheduler to the optimizer
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1368
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1026


Epoch,Training Loss,Validation Loss,Precision Micro,Precision Macro,Precision Weighted,Recall Micro,Recall Macro,Recall Weighted,F1 Micro,F1 Macro,F1 Weighted
1,No log,0.508823,0.776524,0.765434,0.785060,0.955556,0.937719,0.955556,0.856787,0.839669,0.858732
2,0.566500,0.450541,0.800464,0.795882,0.810750,0.958333,0.949963,0.958333,0.872314,0.862706,0.875319
3,0.566500,0.385774,0.839416,0.838704,0.844452,0.958333,0.949832,0.958333,0.894942,0.889439,0.896427
4,0.399200,0.365551,0.844363,0.841929,0.849324,0.956944,0.952206,0.956944,0.897135,0.892051,0.898409
5,0.250100,0.335137,0.866083,0.863750,0.869401,0.961111,0.957269,0.961111,0.911126,0.907495,0.912351
6,0.250100,0.336843,0.871338,0.863842,0.876564,0.950000,0.947493,0.950000,0.908970,0.901942,0.910427


***** Running Evaluation *****
  Num examples = 142
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-171
Configuration saved in ./results/checkpoint-171/config.json
Model weights saved in ./results/checkpoint-171/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-171/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-171/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 142
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-342
Configuration saved in ./results/checkpoint-342/config.json
Model weights saved in ./results/checkpoint-342/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-342/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-342/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 142
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-513
Configuration saved in ./results/checkpoint-513/config.json
Model weights

TrainOutput(global_step=1026, training_loss=0.35479169700577945, metrics={'train_runtime': 8119.4502, 'train_samples_per_second': 1.011, 'train_steps_per_second': 0.126, 'total_flos': 1087389359751168.0, 'train_loss': 0.35479169700577945, 'epoch': 6.0})

In [26]:
trainer_predictions = trainer.predict(encoded_test)
predictions, labels = trainer_predictions.predictions, trainer_predictions.label_ids
predictions




***** Running Prediction *****
  Num examples = 142
  Batch size = 8


array([[ 3.7454016 ,  4.1052113 ,  4.6784573 ,  4.438225  ,  4.73214   ,
         4.8257036 ,  4.8248854 ],
       [-2.1106007 ,  5.4354463 ,  3.280226  ,  4.356098  ,  5.252583  ,
         4.8234525 ,  4.6886    ],
       [-3.7343805 , -5.265427  ,  1.6158755 ,  4.0526834 ,  3.6060448 ,
        -2.333077  ,  4.174444  ],
       [-2.564696  ,  4.0956993 ,  4.663561  ,  4.674085  ,  5.328754  ,
         4.7195163 ,  4.3739038 ],
       [ 3.8299174 ,  3.4993596 ,  3.7197702 , -3.8134313 , -3.0991077 ,
         4.4262376 ,  3.744118  ],
       [ 3.9896536 ,  4.421682  ,  4.0429134 ,  4.570067  ,  4.4950967 ,
         4.714642  ,  4.743285  ],
       [ 4.331446  , -4.244638  ,  4.3148527 , -3.3090582 ,  1.0205169 ,
        -0.39430267,  3.4751313 ],
       [ 4.1567883 , -3.5048103 ,  4.0904675 , -2.488441  , -2.5104344 ,
         4.3220806 ,  3.8755233 ],
       [-3.2319086 ,  3.1859298 ,  3.8897822 ,  4.2513285 ,  4.3565507 ,
        -2.2275558 ,  4.027837  ],
       [ 3.908672  , -3.2531

In [27]:
# calculate the ROC AUC score
micro_roc_auc = roc_auc_score(labels, predictions, multi_class='ovr', average='micro')
macro_roc_auc = roc_auc_score(labels, predictions, multi_class='ovr', average='macro')
weighted_roc_auc = roc_auc_score(labels, predictions, multi_class='ovr', average='weighted')

print(f"ROC AUC score micro: {micro_roc_auc}")
print(f"ROC AUC score macro: {macro_roc_auc}")
print(f"ROC AUC score weighted: {weighted_roc_auc}")


ROC AUC score micro: 0.9190642741281426
ROC AUC score macro: 0.8500172025221896
ROC AUC score weighted: 0.8271320653146585


In [24]:
writer.close()